In [65]:
from random import randint, choice

# Step 1: Define the finite field GF(2^5)
F.<a> = GF(2^5)
print("Irreducible polynomial:", F.modulus())

# Step 2: Define the polynomial ring over GF(2) and create a random polynomial
R.<x> = GF(2)[]

f = sum(R(randint(0,1)) * x^i for i in range(31))
print("Random polynomial f in GF(2)[x]:", f)


def int_to_field_poly(i, n = 5):
    # Represent integer i as an element of GF(2^n) using binary expansion
    return sum(((i >> j) & 1) * a**j for j in range(n))

def field_poly_to_int(element, n = 5):
    """
    Convert an element from GF(2^n) into an integer,
    interpreting its polynomial representation (in the generator 'a')
    as a binary number.
    
    Parameters:
      element: an element of GF(2^n)
      n: the degree of the extension (here 5)
    
    Returns:
      An integer corresponding to the binary number formed by the coefficients.
    """
    # Get the polynomial representation of the element.
    # The polynomial is in GF(2)[x] and typically represented in ascending order.
    poly = element.polynomial()
    coeffs = poly.list()  # returns list of coefficients [c0, c1, ..., ck]
    # Interpret these coefficients as binary digits: c0 + c1*2 + c2*2^2 + ...
    result = sum(int(c) * (2**i) for i, c in enumerate(coeffs))
    return result

# Step 3: Build the truth table by evaluating f at each element of F
# Sort the elements of F by their integer representation
sorted_F = sorted(list(F), key=lambda c: field_poly_to_int(c))
# Now build the truth table using the sorted elements
truth_table = [f(c) for c in sorted_F]


print("Truth table:")
for i, val in enumerate(truth_table):
    print((i,int_to_field_poly(i)), "->", val)

# Step 4: Modify one value in the truth table, setting it to the field element 1
# First, find indices where the output is not already 1
indices = [i for i, val in enumerate(truth_table) if val != F(1)]
if indices:
    index_to_modify = choice(indices)
else:
    # In the unlikely event that all values are 1, choose index 0.
    index_to_modify = 0

old_val = truth_table[index_to_modify]
truth_table_modified = list(truth_table)
truth_table_modified[index_to_modify] = F(1)
print("\nModified truth table: Changed entry at index", index_to_modify, "from", old_val, "to", F(1))
for i, val in enumerate(truth_table_modified):
    print(i, "->", val)


Irreducible polynomial: x^5 + x^2 + 1
Random polynomial f in GF(2)[x]: x^30 + x^27 + x^26 + x^24 + x^22 + x^21 + x^20 + x^18 + x^17 + x^16 + x^13 + x^12 + x^10 + x^7 + x^6 + x^5 + x^3 + x^2 + 1
Truth table:
(0, 0) -> 1
(1, 1) -> 1
(2, a) -> a^3 + a^2 + a + 1
(3, a + 1) -> a^2 + 1
(4, a^2) -> a^4 + a^3 + a^2 + a + 1
(5, a^2 + 1) -> a^4 + 1
(6, a^2 + a) -> a^4 + a^3 + a^2 + a
(7, a^2 + a + 1) -> 1
(8, a^3) -> a^4 + a^3 + a^2 + a
(9, a^3 + 1) -> a^2
(10, a^3 + a) -> a^4 + a + 1
(11, a^3 + a + 1) -> a^4
(12, a^3 + a^2) -> a^4 + a^3 + a
(13, a^3 + a^2 + 1) -> a^3 + 1
(14, a^3 + a^2 + a) -> a^3
(15, a^3 + a^2 + a + 1) -> a^3 + a^2 + 1
(16, a^4) -> a^4 + a
(17, a^4 + 1) -> a^3 + a^2
(18, a^4 + a) -> a
(19, a^4 + a + 1) -> a^3 + a^2 + a
(20, a^4 + a^2) -> a^4 + a + 1
(21, a^4 + a^2 + 1) -> 1
(22, a^4 + a^2 + a) -> a^3 + a
(23, a^4 + a^2 + a + 1) -> 1
(24, a^4 + a^3) -> 1
(25, a^4 + a^3 + 1) -> a^3 + a^2 + a
(26, a^4 + a^3 + a) -> a + 1
(27, a^4 + a^3 + a + 1) -> a^3 + a + 1
(28, a^4 + a^3 + a^

In [66]:
def polynomial_from_truth_table(truth_table, n=5):
    """
    Given a truth table (a list of values in GF(2^n)) for a function 
    F: GF(2^n) -> GF(2^n) in sorted order (by the integer representation of the field elements),
    returns the unique polynomial p(x) in GF(2^n)[x] that represents the function.
    
    Parameters:
        truth_table (list): A list of 2^n elements from GF(2^n) representing F(x) for x in GF(2^n).
        n (int): The exponent defining the field GF(2^n). Default is 5.
    
    Returns:
        p (polynomial): The polynomial in GF(2^n)[x] that interpolates the truth table.
    """
    # Create the list of domain elements: we assume the truth table is ordered by the integer interpretation.
    xs = [int_to_field_poly(i, n) for i in range(2**n)]
    
    # Pair up each domain element with its corresponding output.
    points = list(zip(xs, truth_table))
    
    # Define the polynomial ring over the field F (which is assumed to be defined as GF(2^n))
    P.<x> = PolynomialRing(F)
    
    # Compute the Lagrange interpolation polynomial through the given points.
    poly = P.lagrange_polynomial(points)
    return poly



# Now, find the polynomial that represents the function given by the truth table.
recovered_poly = polynomial_from_truth_table(truth_table, n=5)
print("\nRecovered interpolation polynomial in GF(2^5)[x]:")
print(recovered_poly)


recovered_poly = polynomial_from_truth_table(truth_table_modified, n=5)
print("\nRecovered interpolation polynomial for modified function in GF(2^5)[x]:")
print(recovered_poly)


Recovered interpolation polynomial in GF(2^5)[x]:
x^30 + x^27 + x^26 + x^24 + x^22 + x^21 + x^20 + x^18 + x^17 + x^16 + x^13 + x^12 + x^10 + x^7 + x^6 + x^5 + x^3 + x^2 + 1

Recovered interpolation polynomial for modified function in GF(2^5)[x]:
a^4*x^31 + (a^4 + a^3 + a + 1)*x^30 + (a^4 + a^3 + a^2 + 1)*x^29 + (a^2 + a)*x^28 + (a^4 + a^3 + a^2 + a + 1)*x^27 + a^3*x^26 + a^3*x^25 + (a^3 + a^2)*x^24 + (a^4 + a^3 + a^2)*x^23 + a*x^22 + (a^3 + a^2 + a)*x^21 + (a^4 + a^2 + a + 1)*x^20 + a^2*x^19 + (a^4 + a^2 + 1)*x^18 + (a^3 + a^2 + a + 1)*x^17 + (a^4 + a)*x^16 + (a^4 + a^2 + 1)*x^15 + (a^3 + a + 1)*x^14 + (a + 1)*x^13 + (a^3 + a + 1)*x^12 + (a^2 + a + 1)*x^11 + (a^4 + a^3 + a)*x^10 + (a^4 + a^3)*x^9 + (a^4 + a^2 + a + 1)*x^8 + a^2*x^6 + a^4*x^5 + (a^4 + a^3 + a^2 + a + 1)*x^4 + (a^3 + a^2 + 1)*x^3 + (a^4 + a^3)*x^2 + (a^4 + a)*x + 1


In [ ]:
def polynomial_from_truth_table(truth_table, n=5):
    """
    Given a truth table (a list of field elements in GF(2^n)) for a function 
    F: GF(2^n) -> GF(2^n), where the inputs are in sorted order (by their integer representation),
    returns the unique interpolation polynomial p(x) in GF(2^n)[x] that represents the function.
    
    Parameters:
        truth_table (list): A list of 2^n elements from GF(2^n) representing F(x) for each x in GF(2^n).
        n (int): The exponent defining the field GF(2^n). Default is 5.
    
    Returns:
        p (polynomial): The polynomial in GF(2^n)[x] that interpolates the truth table.
    """
    # Create the domain: convert each integer 0,1,...,2^n-1 to a field element via its binary expansion.
    xs = [int_to_field_poly(i, n) for i in range(2**n)]
    
    # Pair each domain element with its corresponding output from the truth table.
    points = list(zip(xs, truth_table))
    
    # Define the polynomial ring over F.
    P.<x> = PolynomialRing(F)
    
    # Compute and return the unique interpolation polynomial via Lagrange interpolation.
    poly = P.lagrange_polynomial(points)
    return poly
